In [1]:
!nvidia-smi

Sat Jun 25 20:50:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:4F:00.0 Off |                    0 |
| N/A   35C    P0    82W / 300W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  On   | 00000000:52:00.0 Off |                    0 |
| N/A   

In [2]:
!pip install unseal -q

In [3]:
!pip install transformers -q

In [4]:
from typing import Union, Tuple, Optional

import matplotlib.pyplot as plt
from matplotlib import cm
from statistics import stdev
from matplotlib.colors import BoundaryNorm
import numpy as np
import random
import json
import torch
from tqdm.notebook import tqdm
import unseal
from unseal.hooks import Hook, HookedModel, common_hooks
from unseal.transformers_util import load_from_pretrained, get_num_layers

import sys
sys.path.append("../../lib")
import utility
import hooks

In [5]:
unhooked_model, tokenizer, config = load_from_pretrained('EleutherAI/gpt-j-6B')

In [6]:
model = HookedModel(unhooked_model)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print(f'Model loaded and on device {device}!')

Model loaded and on device cuda!


In [7]:
# layer names
print(list(model.layers.keys()))

['transformer', 'transformer->wte', 'transformer->drop', 'transformer->h', 'transformer->h->0', 'transformer->h->0->ln_1', 'transformer->h->0->attn', 'transformer->h->0->attn->attn_dropout', 'transformer->h->0->attn->resid_dropout', 'transformer->h->0->attn->k_proj', 'transformer->h->0->attn->v_proj', 'transformer->h->0->attn->q_proj', 'transformer->h->0->attn->out_proj', 'transformer->h->0->mlp', 'transformer->h->0->mlp->fc_in', 'transformer->h->0->mlp->fc_out', 'transformer->h->0->mlp->act', 'transformer->h->0->mlp->dropout', 'transformer->h->1', 'transformer->h->1->ln_1', 'transformer->h->1->attn', 'transformer->h->1->attn->attn_dropout', 'transformer->h->1->attn->resid_dropout', 'transformer->h->1->attn->k_proj', 'transformer->h->1->attn->v_proj', 'transformer->h->1->attn->q_proj', 'transformer->h->1->attn->out_proj', 'transformer->h->1->mlp', 'transformer->h->1->mlp->fc_in', 'transformer->h->1->mlp->fc_out', 'transformer->h->1->mlp->act', 'transformer->h->1->mlp->dropout', 'transf

In [8]:
# load data
f = open("../../datasets/known_1000.json")
prompts = json.load(f)

In [9]:
# prepare data
texts = [(prompts[i]['subject'], prompts[i]['prompt'] + ' ' + prompts[i]['attribute']) for i in range(len(prompts))]
print(texts[0])

('Vinson Massif', 'Vinson Massif is located in the continent of Antarctica')


In [10]:
### Batching requires inputs to be of the same size so we put each prompt in 
### a bucket depending on the number of tokens it contains.

inputs = [[] for z in range(30)] ## use 30 as an upper bound on the number of tokens
for subject, text in random.sample(texts, 1000):
    encoded_text, target_id = utility.prepare_input(text, tokenizer, device)
    num_tokens = encoded_text['input_ids'].shape[1]
    
    print("num tokens: ", num_tokens, "text: ", text)
    
    # get tokens
    input_ids = encoded_text['input_ids'][0]
    tkens = [tokenizer.decode(input_ids[i]) for i in range(len(input_ids))]
    
    # get position of subject tokens
    subject_positions = utility.get_subject_positions(tkens, subject)
    indx_str = str(subject_positions[0]) + ":" + str(subject_positions[-1]+1)
    
    # add star to subject tokens to later do analysis
    start, end = subject_positions[0], subject_positions[1]
    while start < end:
        tkens[start] = tkens[start] + "*"
        start += 1
    
    # save uncorrupted states for patching
    num_layers = get_num_layers(model, layer_key_prefix='transformer->h')
    output_hooks_names = [f'transformer->h->{layer}' for layer in range(num_layers)]
    output_hooks = [Hook(name, utility.save_output, name) for name in output_hooks_names]
    model(**encoded_text, hooks=output_hooks)
    
    hidden_states = [model.save_ctx[f'transformer->h->{layer}']['output'][0].detach() for layer in range(num_layers)] # need detach
    
    # get probability with uncorrupted subject
    p = model(**encoded_text, hooks=[])['logits'].softmax(dim=-1)[0,-1,target_id].item()
    
    NUM_RUNS = 10 # ROME runs each input 10 times
    for i in range(NUM_RUNS):
        # add noise hook
        seed = np.random.randint(100, size=1)[0]
        noise_hook = Hook(
            layer_name='transformer->wte',
            func=hooks.additive_output_noise(indices=indx_str, mean=0, std=0.025, seed=seed),
            key='embedding_noise',
        )
    
        # get probability with corrupted subject
        p_star = model(**encoded_text, hooks=[noise_hook])['logits'].softmax(dim=-1)[0,-1,target_id].item()
        
        inputs[num_tokens].append((num_tokens, num_layers, encoded_text, hidden_states, tkens, text, target_id, indx_str, seed, p, p_star))


num tokens:  11 text:  Valdemarsvik is located in the country of Sweden
num tokens:  9 text:  Nikon formed in 1975 and is headquartered in Tokyo
num tokens:  5 text:  Chanel was started in Paris
num tokens:  8 text:  Paul Morand died in the city of Paris
num tokens:  6 text:  Edina High School, in Minnesota
num tokens:  13 text:  Chakrapani was created in the country of his birth, India
num tokens:  11 text:  In Ruokolahti, the language spoken is Finnish
num tokens:  16 text:  Buenos Aires Herald, created in 1891, is the oldest daily newspaper in Argentina
num tokens:  11 text:  Grossman Nunataks belongs to the continent of Antarctica
num tokens:  8 text:  Italian Football Federation is a part of the FIFA
num tokens:  14 text:  Leo XIII, whose position is that of a "superior" pope
num tokens:  17 text:  The language used by Jean-Baptiste Marchand is not the language of the French
num tokens:  9 text:  X11.app, a product developed by Apple
num tokens:  6 text:  Leonard Bernstein perform

num tokens:  17 text:  Elizabeth Butler-Sloss, Baroness Butler-Sloss worked in the city of London
num tokens:  8 text:  The native language of Michel Modo is French
num tokens:  6 text:  Adam Kendon was born in London
num tokens:  6 text:  Acura EL is developed by Honda
num tokens:  9 text:  Boniface III originated from the city of Rome
num tokens:  11 text:  Inoue Genan Inseki is native to Japan
num tokens:  9 text:  Eurovision Song Contest 1960 premieres on the BBC
num tokens:  12 text:  In Aarau, the language spoken is a mixture of German
num tokens:  7 text:  Shane McMahon is employed by the WWE
num tokens:  7 text:  David Suzuki Foundation's headquarters are in Vancouver
num tokens:  10 text:  public transport in Istanbul is located in the country of Turkey
num tokens:  7 text:  Catalonia belongs to the continent of Europe
num tokens:  7 text:  Windows Internet Explorer 8 is developed by Microsoft
num tokens:  8 text:  Hideki Shirakawa was born in Tokyo
num tokens:  16 text:  Giov

num tokens:  6 text:  The capital of Roman Republic is Rome
num tokens:  9 text:  Debenham Glacier belongs to the continent of Antarctica
num tokens:  8 text:  L'Aurore is written in French
num tokens:  12 text:  Pope Heraclas of Alexandria has the position of the first bishop
num tokens:  7 text:  The Hague belongs to the continent of Europe
num tokens:  10 text:  The capital city of United Kingdom of the Netherlands is Amsterdam
num tokens:  15 text:  Eusebius of Vercelli, who has the position of a bishop
num tokens:  10 text:  John Smoltz plays in the position of the pitcher
num tokens:  8 text:  Cape Adams belongs to the continent of Antarctica
num tokens:  13 text:  In Seychelles, the language spoken is a mix of English
num tokens:  10 text:  Lev Vygotsky works in the field of psychology
num tokens:  14 text:  David Crystal's area of work is the study of the history of the English
num tokens:  10 text:  Galileo Galilei works in the area of astronomy
num tokens:  5 text:  Norway's 

num tokens:  7 text:  Kingdom of Egypt's capital, Cairo
num tokens:  11 text:  Avenue Montaigne was created in the country of France
num tokens:  17 text:  Rosenberg Trio is known for performing a variety of styles of music, including jazz
num tokens:  8 text:  The location of Harvard Law School is in Cambridge
num tokens:  13 text:  The native language of Henriette-Julie de Murat is French
num tokens:  8 text:  Lawrence Taylor professionally plays the sport of football
num tokens:  10 text:  Julian Huxley died in the city of London
num tokens:  6 text:  Alan Sugar is employed by the BBC
num tokens:  14 text:  North Side Gang was created in the early 1980s by a group of Chicago
num tokens:  7 text:  Yakuza 2 is developed by Sega
num tokens:  6 text:  Windows Live Mail is developed by Microsoft
num tokens:  8 text:  Seiyu Group's headquarters are in Tokyo
num tokens:  10 text:  The language of Dwynwen is a mixture of Welsh
num tokens:  7 text:  Mark Sanchez plays in the position of quar

num tokens:  10 text:  Rutland Barrington died in the city of London
num tokens:  15 text:  Bhaktisiddhanta Saraswati, who has a citizenship of India
num tokens:  8 text:  Angela Ahrendts is employed by Apple
num tokens:  6 text:  A Charlie Brown Christmas debuted on CBS
num tokens:  9 text:  Alvin and the Chipmunks premiered on NBC
num tokens:  12 text:  The language used by Musa Manarov is a mixture of Russian
num tokens:  16 text:  Antonio Sartorio is known for performing in the world's most prestigious opera
num tokens:  10 text:  Yannick Cahuzac plays as a central midfielder
num tokens:  10 text:  In Indiana, the language spoken is a mixture of English
num tokens:  12 text:  Isola Dovarese is located in the country of Italy
num tokens:  10 text:  Frederick Banting specializes in the study of the physiology
num tokens:  9 text:  EDP Renewables North America is headquartered in Houston
num tokens:  5 text:  Chris Cuomo is employed by CNN
num tokens:  12 text:  Yitzhak Ben-Zvi worked

num tokens:  10 text:  Farrukhsiyar follows the religion of Islam
num tokens:  5 text:  Bandai Channel started in Japan
num tokens:  6 text:  Canadian Cancer Society is headquartered in Toronto
num tokens:  12 text:  The original language of Plus belle la vie is a French
num tokens:  8 text:  Schreckhorn belongs to the continent of Europe
num tokens:  7 text:  Steve Claridge is employed by the BBC
num tokens:  6 text:  Jeff Hardy is affiliated with the WWE
num tokens:  14 text:  Pius VI, whose position is that of a "superior" pope
num tokens:  8 text:  David Mellor worked in the city of London
num tokens:  7 text:  Jeep Grand Cherokee is developed by Chrysler
num tokens:  9 text:  The capital city of Kingdom of the Netherlands is Amsterdam
num tokens:  8 text:  Don Shula professionally plays the sport of football
num tokens:  8 text:  Farouk of Egypt follows the religion of Islam
num tokens:  7 text:  Trailer Park Boys premieres on Netflix
num tokens:  9 text:  The native language of G

num tokens:  12 text:  Christian Medical College Ludhiana is located in the country of India
num tokens:  6 text:  Internet Explorer 10 is developed by Microsoft
num tokens:  9 text:  Shetland Islands, in the north of Scotland
num tokens:  7 text:  France Dimanche is written in the French
num tokens:  4 text:  Xbox, created by Microsoft
num tokens:  10 text:  Renata Polverini worked in the city of Rome
num tokens:  9 text:  William Bayliss died in the city of London
num tokens:  4 text:  iPhone is developed by Apple
num tokens:  10 text:  Ne partez pas sans moi is written in French
num tokens:  9 text:  Edmund Neupert, performing on the piano
num tokens:  14 text:  Clement VIII, whose position is that of a "superior" pope
num tokens:  14 text:  East Japan Railway Company, that was formed in 1894, and the Tokyo
num tokens:  17 text:  The language used by Ch'ien Mu is very similar to the language used by the Chinese
num tokens:  7 text:  ActiveSync, a product developed by Microsoft
num t

num tokens:  7 text:  Second Polish Republic's capital city is Warsaw
num tokens:  7 text:  Jessica Jones is a member of the Avengers
num tokens:  11 text:  The location of 2000 Australian Open is in the heart of Melbourne
num tokens:  11 text:  Hebrew Theological College is affiliated with the religion of Judaism
num tokens:  11 text:  In Marshall Islands, the language spoken is a mixture of English
num tokens:  5 text:  Big Love premieres on HBO
num tokens:  11 text:  The capital of Fatimid caliphate is the city of Cairo
num tokens:  10 text:  Jan Machulski, who has a citizenship of Poland
num tokens:  7 text:  Wii MotionPlus is developed by Nintendo
num tokens:  9 text:  Jules Moch worked in the city of Paris
num tokens:  8 text:  Italy national football team is affiliated with the FIFA
num tokens:  8 text:  Tim Tebow plays in the position of quarterback
num tokens:  11 text:  North Middlesex, Ontario is located in the country of Canada
num tokens:  10 text:  Louis XVII of France di

num tokens:  8 text:  Daiki Arioka's profession is an actor
num tokens:  12 text:  The original language of Yalkut Yosef was written in Hebrew
num tokens:  7 text:  Epiphone's owner, the Gibson
num tokens:  18 text:  Peter Josef von Lindpaintner is known for performing the first ever performance of the opera
num tokens:  9 text:  Delia Derbyshire is employed by the BBC
num tokens:  14 text:  The original language of Tusculanae Disputationes is a Latin
num tokens:  9 text:  The headquarter of Canada West Foundation is in Calgary
num tokens:  9 text:  How to Make It in America premieres on HBO
num tokens:  8 text:  Giacinto Brandi is originally from Italy
num tokens:  9 text:  Clifford Curzon, performing on the piano
num tokens:  13 text:  Ludovic Sylvestre plays in the position of a defensive midfielder
num tokens:  10 text:  The original language of Revue de Paris is a French
num tokens:  11 text:  Liquor Control Board of Ontario's headquarters are in Toronto
num tokens:  10 text:  Kin

In [11]:
# statistics on number of prompts for each length tokens
for i in range(len(inputs)):
    length = len(inputs[i])
    if length != 0:
        print("num tokens: ", i, "number of prompts: ", int(length/NUM_RUNS))

num tokens:  4 number of prompts:  11
num tokens:  5 number of prompts:  40
num tokens:  6 number of prompts:  100
num tokens:  7 number of prompts:  120
num tokens:  8 number of prompts:  133
num tokens:  9 number of prompts:  126
num tokens:  10 number of prompts:  134
num tokens:  11 number of prompts:  104
num tokens:  12 number of prompts:  57
num tokens:  13 number of prompts:  41
num tokens:  14 number of prompts:  51
num tokens:  15 number of prompts:  22
num tokens:  16 number of prompts:  16
num tokens:  17 number of prompts:  18
num tokens:  18 number of prompts:  15
num tokens:  19 number of prompts:  3
num tokens:  20 number of prompts:  2
num tokens:  21 number of prompts:  2
num tokens:  22 number of prompts:  1
num tokens:  23 number of prompts:  2
num tokens:  25 number of prompts:  1
num tokens:  27 number of prompts:  1


In [12]:
# remove input buckets with 0 prompts
inputs = [inp for inp in inputs if inp != []]

In [13]:
# deal with longest prompts first :)
inputs.reverse()

In [14]:
from torch.utils.data import DataLoader

results = {}
batch_schedule = {i : int(1000/i) for i in range(1, 30)}
for inp in inputs:
    print(f"Number of prompts in this bucket: {len(inp)}.")
    
    batch_size = batch_schedule[inp[0][0]]
    batched_data = DataLoader(inp, batch_size=batch_size)
    for data in batched_data:
        curr_batch_size = data[0].size()[0]
        print("This batch has", curr_batch_size, "examples.")
        
        num_tokens = data[0][0].item()
        num_layers = data[1][0].item()
        encoded_text = data[2]
        hidden_states = data[3]
        tkens = [[t[i] for t in data[4]] for i in range(curr_batch_size)]
        text = [t for t in data[5]]
        target_ids = [t.item() for t in data[6]]
        indx_strs = [s for s in data[7]]
        seeds = [seed.item() for seed in data[8]]
        p = [p.item() for p in data[9]]
        p_stars = [p_s.item() for p_s in data[10]]

        noise_hooks = [] # can probably be batched [will take a look later on]
        for i in range(curr_batch_size):
            noise_hook = Hook(
                layer_name='transformer->wte',
                func=hooks.additive_output_noise(indices=indx_strs[i], mean=0, std=0.025, index=i, seed=seeds[i]),
                key='embedding_noise',
            )
            noise_hooks.append(noise_hook)
            
        for layer in tqdm(range(num_layers)):
            for position in range(num_tokens):
                hook = Hook(
                    layer_name=f'transformer->h->{layer}',
                    func= hooks.hidden_patch_hook_fn(layer, position, hidden_states),
                    key=f'patch_{layer}_pos{position}'
                )
                output = model(**encoded_text, hooks=noise_hooks+[hook])
                for i in range(curr_batch_size):
                    if text[i] not in results:
                        results[text[i]] = (torch.zeros((num_tokens, num_layers)), (text[i], tkens[i], num_layers, sum(p) / len(p), sum(p_stars) / len(p_stars)))
                    results[text[i]][0][position, layer] += torch.softmax(output["logits"][i][0,-1,:], 0)[target_ids[i]].item() - p_stars[i]

Number of prompts in this bucket: 10.
This batch has 10 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

Number of prompts in this bucket: 10.
This batch has 10 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

Number of prompts in this bucket: 20.
This batch has 20 examples.


...

  0%|          | 0/28 [00:00<?, ?it/s]

This batch has 200 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

Number of prompts in this bucket: 110.
This batch has 110 examples.


  0%|          | 0/28 [00:00<?, ?it/s]

In [17]:
# Need to divide by NUM_RUNS
res = []
for text, value in results.items():
    result, access = value
    result = result / NUM_RUNS
    res.append((result, access))

In [18]:
torch.save(res, "indirect_effect_1000_examples.pt")